# User Workspace

This section completes the tutorial workflow by:
* Registering the user's processing results in their workspace
* Inspecting their product metadata via their Resource Catalogue
* Visualising their results via the Data Access services

In [ ]:
import requests
import datetime

In [ ]:
domain = "demo.eoepca.org"
#domain = "develop.eoepca.org"
workspace_prefix = "demo-user"

In [ ]:
USERNAME="eric"
PASSWORD="defaultPWD"

In [ ]:
workspaces_endpoint = f"https://workspace-api.{domain}/workspaces"

In [ ]:
import utils.DemoClient as client
import jwt

demo = client.DemoClient(f"https://auth.{domain}")
demo.register_client()
demo.save_state()

In [ ]:
user_id_token = demo.get_id_token(USERNAME, PASSWORD)
headers = {
    'Authorization': 'Bearer ' + user_id_token
}

jwt_header = jwt.get_unverified_header(user_id_token)
jwt_payload = jwt.decode(user_id_token, options={"verify_signature": False})
user_uid = jwt_payload["sub"]

In [ ]:
actual_workspace_name = f'{workspace_prefix}-{USERNAME}'
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}", headers=headers)
response.raise_for_status()

workspace_data = response.json()
bucket_name = workspace_data["storage"]["credentials"]["bucketname"]
s3_access = workspace_data["storage"]["credentials"]["access"]
s3_secret = workspace_data["storage"]["credentials"]["secret"]
workspace_data["storage"]["credentials"]['secret'] = "hidden_for_demo"
workspace_data["container_registry"]["password"] = "hidden_for_demo"
workspace_data

We will now upload an application package to the workspace:

In [ ]:
import boto3
#S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
S3_ENDPOINT = "https://minio.demo.eoepca.org"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)
object = s3resource.Object(bucket_name, 'application-package/s-expression/s-expression-0_0_2.cwl')
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

Now we register the application package:

In [ ]:
resource_url= f'{bucket_name}/application-package/s-expression/s-expression-0_0_2.cwl'
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register",
    headers=headers,
    json={
        "type": "application",
        "url": resource_url,
    }
)
response.raise_for_status()
response

### Workspace catalogue

In [ ]:
from owslib.csw import CatalogueServiceWeb

In [ ]:
workspace_catalogue_endpoint = f'https://resource-catalogue.{actual_workspace_name}.{domain}'
workspace_catalogue_csw = f'{workspace_catalogue_endpoint}/csw'

Now we check the registration result:

In [ ]:
import time
time.sleep(3)
response = requests.get(f"{workspace_catalogue_endpoint}/collections/metadata:main/items?type=application", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

In [ ]:
headers = {
    'Authorization': 'Bearer ' + user_id_token
}

List all records identifier, type and title in workspace catalogue

In [ ]:
csw = CatalogueServiceWeb(workspace_catalogue_csw, timeout=30,headers=headers)
csw.getrecords2(maxrecords=10)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

In [ ]:
csw.records['s-expression'].references